In [ ]:
pip install together PyMuPDF tiktoken pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.3 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import tiktoken

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

def chunk_text(text, chunk_size=1000):
    """Splits text into manageable chunks based on token limits."""
    tokenizer = tiktoken.get_encoding("cl100k_base")  # OpenAI tokenizer
    tokens = tokenizer.encode(text)

    chunks = []
    for i in range(0, len(tokens), chunk_size):
        chunk = tokens[i:i+chunk_size]
        chunks.append(tokenizer.decode(chunk))

    return chunks

# Extract and split PDF text
pdf_path = "/content/data.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
text_chunks = chunk_text(extracted_text, chunk_size=1000)

print(f"PDF split into {len(text_chunks)} chunks.")


PDF split into 30 chunks.


In [ ]:
college_data_chunks = {i: chunk for i, chunk in enumerate(text_chunks)}

# Save for future use
import pickle

with open("justice_model.pkl", "wb") as f:
    pickle.dump(college_data_chunks, f)

print("Justice data saved in chunks for retrieval.")

Justice data saved in chunks for retrieval.


In [ ]:
import json
from together import Together
import fitz  # PyMuPDF

# Load and chunk the Justice PDF
def extract_pdf_chunks(pdf_path, chunk_size=1000):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()

    # Split into chunks of `chunk_size` characters
    return [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]

# Extracted chunks from the justice report
college_data_chunks = extract_pdf_chunks("data.pdf")

# Initialize Together API client
client = Together(api_key="8171eae9a74b4d63a72d2b802c4c13105e8daba6b173fd0e6ca5bbcef9bc1e4d")  # Replace with your actual API key

# Retrieval function
def retrieve_relevant_chunk(query):
    relevant_chunks = []
    for chunk in college_data_chunks:
        if query.lower() in chunk.lower():
            relevant_chunks.append(chunk)
    return " ".join(relevant_chunks[:3])  # Limit to 3 chunks

# Main query function
def query_justice_report(query):
    relevant_data = retrieve_relevant_chunk(query)

    prompt = f"""
    You are an expert in Indian judicial governance. A user asked: "{query}"

    Here is relevant context from an official government document:
    {relevant_data}

    Based on the above, provide a clear and concise answer.
    """

    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content.strip()

# Example usage
user_query = "What is the role of Department of Justice in judicial appointments?"
print(query_justice_report(user_query))

The Department of Justice plays a crucial role in the judicial appointments process in India. Although the specific context is not provided, generally, the Department of Justice is responsible for facilitating the appointment of judges to the higher judiciary, including the Supreme Court and High Courts.

The Department's role typically involves:

1. Processing recommendations: The Department receives recommendations from the Supreme Court Collegium or High Court Collegiums for appointments.
2. Verifying credentials: It verifies the credentials, antecedents, and suitability of the recommended candidates.
3. Obtaining IB reports: The Department obtains reports from the Intelligence Bureau (IB) on the candidates, which are then considered by the Collegium.
4. Coordination with Ministries: The Department coordinates with other Ministries, such as the Ministry of Law and Justice, to ensure a smooth appointment process.
5. Notification of appointments: Once the appointments are finalized, t

In [ ]:
import json
from together import Together

# Load justice data from JSON file
with open("justice_data.json", "r") as f:
    justice_data = json.load(f)

# Initialize Together API client
client = Together(api_key="8171eae9a74b4d63a72d2b802c4c13105e8daba6b173fd0e6ca5bbcef9bc1e4d")  # Replace with your API key

# Retrieve matching chunks from JSON content
def retrieve_relevant_chunk(query):
    relevant_chunks = []

    for topic, entries in justice_data.items():
        for item in entries:
            if query.lower() in item.lower() or query.lower() in topic.lower():
                relevant_chunks.append(f"{topic}: {item}")

    return " ".join(relevant_chunks[:5])  # Limit to 5 matching entries

# Query function using Together's LLM
def query_justice_json(query):
    relevant_data = retrieve_relevant_chunk(query)

    prompt = f"""
    You are a legal expert familiar with India's Department of Justice.

    Based on the following official information:
    {relevant_data}

    Answer the user's query: "{query}"
    """

    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content.strip()

# Example usage
user_query = "What are the guidelines for e-filing court documents?"
print(query_justice_json(user_query))

The guidelines for e-filing court documents in India are outlined by the Department of Justice and the respective courts. Here are the general guidelines:

1. **Registration**: To e-file court documents, you need to register on the court's website or the National Judicial Data Grid (NJDG) portal. You will need to provide basic information such as name, email ID, mobile number, and password.
2. **Document Preparation**: Ensure that the documents to be e-filed are in the prescribed format and contain all the necessary information. The documents should be scanned and saved in PDF format.
3. **Filing Fees**: Pay the required filing fees online through the court's website or the NJDG portal. The fees can be paid using credit/debit cards, net banking, or other online payment modes.
4. **E-Filing Process**: Log in to the court's website or the NJDG portal and select the option for e-filing. Fill in the required details, upload the documents, and pay the filing fees.
5. **Verification**: After